In [461]:
class Cell:
    def __init__(self, initial_value = False):
        if not initial_value:
            self.domain = set([1,2,3,4,5,6,7,8,9])
        else:
            assert initial_value in range(1,10)
            self.domain = set([initial_value])
            
    def remove(self, item):
        if item in self.domain:
            self.domain.remove(item)
            return True
        else:
            return False
        
    def get_known_cell_value(self):
        assert len(self.domain) == 1
        for elem in self.domain:
            return elem
    
    def is_known(self):
        return len(self.domain) == 1

empty_cell = "."
class Sudoku:
    board = [["empty" for _ in range(9)] for _ in range(9)]
    known_indices = set([])
    unknown_indices = set([])
    def __init__(self, list_board):
        #assume '.' denodes empty cells, '4' would be a valued cell
        for row in range(9):
            for col in range(9):
                list_item = list_board[row][col]
                if list_item == empty_cell:
                    self.board[row][col] = Cell()
                    self.unknown_indices.add((row, col))
                else:
                    self.board[row][col] = Cell(int(list_item))
                    self.known_indices.add((row, col))
                    
    def get_cell(self, row, col):
        return self.board[row][col]
                    
    def print_board(self):
        for row in self.board:
            buff = []
            for cell in row:
                if cell.is_known():
                    buff.append(cell.get_known_cell_value())
                else:
                    buff.append(empty_cell)
            print(*buff)
    
    def return_board(self):
        board_list = []
        for row in self.board:
            buff = []
            for cell in row:
                if cell.is_known():
                    buff.append(str(cell.get_known_cell_value()))
                else:
                    buff.append(empty_cell)
            board_list.append(buff)
        return board_list
    
    def row_indices(self, row, col):
        return set([(row_i, col) for row_i in range(9) if row_i != row])
    
    def col_indices(self, row, col):
        return set([(row, col_i) for col_i in range(9) if col_i != col])
        
    def sub_block_indices(self, row, col):
        sub_row = row // 3
        sub_col = col // 3
        to_return = set([(3*sub_row + row_i, 3*sub_col + col_i) for row_i in range(3) for col_i in range(3)])
        to_return.remove((row, col))
        return to_return
    
    def get_constrained_indices(self, row, col):
        return self.row_indices(row, col).union(self.col_indices(row, col)).union(self.sub_block_indices(row, col))
    
    def arc_consistency_outwards(self):
        removed_count = 0
        now_known_indices = set([])
        for known_index in self.known_indices:
            cell = self.get_cell(*known_index)
#             print(cell.domain)
            cell_value = cell.get_known_cell_value()
            for constrained_index in self.unknown_indices.intersection(self.get_constrained_indices(*known_index)):
                constrained_cell = self.get_cell(*constrained_index)
                if constrained_cell.remove(cell_value) == True:
                    removed_count += 1
                    if constrained_cell.is_known():
                        self.unknown_indices.remove(constrained_index)
                        now_known_indices.add(constrained_index)
                        
        self.known_indices = self.known_indices.union(now_known_indices)
        self.unknown_indices.difference_update(now_known_indices)
        return removed_count
    
    def all_different_constraint(self):
        '''ex: if unknown_cell is the only cell in its row that can be a 2, unknown cell must be a 2'''
        now_known_indices = set([])
        for unknown_index in self.unknown_indices:
            unknown_cell = self.get_cell(*unknown_index)
            
            row_domain = set([])
            for row_index in self.row_indices(*unknown_index):
                row_domain = row_domain.union(self.get_cell(*row_index).domain)
            if len(unknown_cell.domain.difference(row_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(row_domain)
                now_known_indices.add(unknown_index)
                continue
                
            col_domain = set([])
            for col_index in self.col_indices(*unknown_index):
                col_domain = col_domain.union(self.get_cell(*col_index).domain)
            if len(unknown_cell.domain.difference(col_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(col_domain)
                now_known_indices.add(unknown_index)
                continue
                
            sub_block_domain = set([])
            for sub_index in self.sub_block_indices(*unknown_index):
                sub_block_domain = sub_block_domain.union(self.get_cell(*sub_index).domain)
            if len(unknown_cell.domain.difference(sub_block_domain)) == 1:
                unknown_cell.domain = unknown_cell.domain.difference(sub_block_domain)
                now_known_indices.add(unknown_index)
                continue
        
        self.known_indices = self.known_indices.union(now_known_indices)
        self.unknown_indices.difference_update(now_known_indices)

In [462]:
test_input = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]
test_input2 = [[".",".","9","7","4","8",".",".","."],["7",".",".",".",".",".",".",".","."],[".","2",".","1",".","9",".",".","."],[".",".","7",".",".",".","2","4","."],[".","6","4",".","1",".","5","9","."],[".","9","8",".",".",".","3",".","."],[".",".",".","8",".","3",".","2","."],[".",".",".",".",".",".",".",".","6"],[".",".",".","2","7","5","9",".","."]]
S = Sudoku(test_input2)
print("Start")
S.print_board()

Start
. . 9 7 4 8 . . .
7 . . . . . . . .
. 2 . 1 . 9 . . .
. . 7 . . . 2 4 .
. 6 4 . 1 . 5 9 .
. 9 8 . . . 3 . .
. . . 8 . 3 . 2 .
. . . . . . . . 6
. . . 2 7 5 9 . .


In [463]:
import time
start = time.time()
prev_known_indices = -1
while len(S.known_indices) != prev_known_indices:
    prev_known_indices = len(S.known_indices)
    print("known:", len(S.known_indices))
    S.print_board()
    S.arc_consistency_outwards()
    S.all_different_constraint()
    
print("completion time (s)", time.time() - start)
S.print_board()

known: 27
. . 9 7 4 8 . . .
7 . . . . . . . .
. 2 . 1 . 9 . . .
. . 7 . . . 2 4 .
. 6 4 . 1 . 5 9 .
. 9 8 . . . 3 . .
. . . 8 . 3 . 2 .
. . . . . . . . 6
. . . 2 7 5 9 . .
known: 40
. . 9 7 4 8 . . 2
7 . . . . . . . 9
. 2 . 1 . 9 . . .
. . 7 . 8 6 2 4 .
. 6 4 3 1 7 5 9 8
. 9 8 . . . 3 6 .
9 . . 8 6 3 . 2 .
. . 2 . 9 1 . . 6
. . . 2 7 5 9 . .
known: 51
. . 9 7 4 8 . . 2
7 . . 6 . 2 . . 9
. 2 . 1 . 9 . . .
. . 7 9 8 6 2 4 1
2 6 4 3 1 7 5 9 8
1 9 8 5 2 4 3 6 7
9 . . 8 6 3 . 2 .
. . 2 4 9 1 . . 6
. . . 2 7 5 9 . .
completion time (s) 0.024727821350097656
. . 9 7 4 8 . . 2
7 . . 6 . 2 . . 9
. 2 . 1 . 9 . . .
. . 7 9 8 6 2 4 1
2 6 4 3 1 7 5 9 8
1 9 8 5 2 4 3 6 7
9 . . 8 6 3 . 2 .
. . 2 4 9 1 . . 6
. . . 2 7 5 9 . .


In [466]:
sorted_indices = [cell_index for cell_index in S.unknown_indices]
sorted_indices.sort()
for sorted_index in sorted_indices:
    cell = S.get_cell(*sorted_index)
    print(sorted_index, cell.domain)

(0, 0) {3, 5, 6}
(0, 1) {1, 3, 5}
(0, 6) {1, 6}
(0, 7) {1, 3, 5}
(1, 1) {1, 3, 4, 5, 8}
(1, 2) {1, 3, 5}
(1, 4) {3, 5}
(1, 6) {1, 4, 8}
(1, 7) {1, 3, 5, 8}
(2, 0) {3, 4, 5, 6, 8}
(2, 2) {3, 5, 6}
(2, 4) {3, 5}
(2, 6) {4, 6, 7, 8}
(2, 7) {3, 5, 7, 8}
(2, 8) {3, 4, 5}
(3, 0) {3, 5}
(3, 1) {3, 5}
(6, 1) {1, 4, 5, 7}
(6, 2) {1, 5}
(6, 6) {1, 4, 7}
(6, 8) {4, 5}
(7, 0) {3, 5, 8}
(7, 1) {3, 5, 7, 8}
(7, 6) {7, 8}
(7, 7) {3, 5, 7, 8}
(8, 0) {3, 4, 6, 8}
(8, 1) {1, 3, 4, 8}
(8, 2) {1, 3, 6}
(8, 7) {1, 3, 8}
(8, 8) {3, 4}
